In [1]:
import pandas as pd

In [2]:
import itertools

In [3]:
import numpy as np 

In [4]:
df_de = pd.read_json('dataset/QuaTrain-de.json', lines=True)

In [25]:
df_de.head()

,text,language,rhyme,meter,alliteration
0,"[Da jener Elbe-Schwan der Gottheit Ruhm, Da Br...",de,ABCD,iambus,0.040973
1,"[Da Brokkes Saitenspiel, so oft das Herze, Das...",de,ABCD,iambus,0.054848
2,"[gen, der belebt Feld, Berg und Thal;, Der erw...",de,ABCD,iambus,0.023747
3,"[Ueber die erhabnen Berge, und erwekt das Feld...",de,ABCD,alexandrine,0.044927
4,[Als mit holden Dünsten füllet; da was aus den...,de,ABCD,other,0.084108


In [6]:
for l in list(df_de["text"])[:15]:
    print (l, "\n")

['Da jener Elbe-Schwan der Gottheit Ruhm', 'Da Brokkes Saitenspiel, so oft das Herze', 'Das jederman gefällt, in heiligs Feuer', 'Wie kan beim hellen Thon, ein heisch und'] 

['Da Brokkes Saitenspiel, so oft das Herze', 'Das jederman gefällt, in heiligs Feuer', 'Wie kan beim hellen Thon, ein heisch und', 'Bei einem Schwang-Gesang ein schnatternd'] 

['gen, der belebt Feld, Berg und Thal;', 'Der erwärmet die Natur, durch', 'Und erhizt bei dem was lebt, ein', 'Eure Anmuth nährt die Triebe,'] 

['Ueber die erhabnen Berge, und erwekt das Feld', 'Welch ein Anblik voller Lust! rühret die erwekten', 'Man sieht in den feuchten Thau, lauter Seegens', 'Auf! entschlafne Erdenbürger! auf ermuntert eur'] 

['Als mit holden Dünsten füllet; da was aus den', 'Wenn die sanften Westen säuseln, wie ein lieblich', 'Und uns den Geruch erquikt. O! ein angenehmes', 'Das die Lebens Geister stärkt, kreucht unsichtbar'] 

['Wenn die sanften Westen säuseln, wie ein lieblich', 'Und uns den Geruch erquikt. O! ein 

In [7]:
def get_unique_quatrains(df):
    last_t = ""
    overlapping = []
    unique_rows = []
    for i, row in df.iterrows():
        if list(row["text"])[0] == last_t:
            overlapping.append(row)
        elif len(overlapping) == 0:
            overlapping = [row]
        else:
            unique_rows.append(overlapping[0])
            overlapping = [row]
        last_t = list(row["text"])[1]
    unique_rows.append(overlapping[0]) 
    #print(unique_rows)   
    df_unique = pd.DataFrame(unique_rows)
    return(df_unique)

In [8]:
df_de_unique = get_unique_quatrains(df_de)

In [9]:
df_de_unique.head()

,text,language,rhyme,meter,alliteration
0,"[Da jener Elbe-Schwan der Gottheit Ruhm, Da Br...",de,ABCD,iambus,0.040973
2,"[gen, der belebt Feld, Berg und Thal;, Der erw...",de,ABCD,iambus,0.023747
3,"[Ueber die erhabnen Berge, und erwekt das Feld...",de,ABCD,alexandrine,0.044927
4,[Als mit holden Dünsten füllet; da was aus den...,de,ABCD,other,0.084108
9,"[Hie brummt eine Käfer Menge da saußt bei der,...",de,ABCD,iambus,0.044130


In [10]:
len(df_de)

1483685

In [11]:
len(df_de_unique)

363698

In [12]:
def check_common_elements(lists):
    for i in range(len(lists)):
        for j in range(i + 1, len(lists)):
            if any(elem in lists[i] for elem in lists[j]):
                return True
    return False

# Test the function
lists = [[1, 2, 3], [4, 5, 6], [7, 8, 9], [8]]
result = check_common_elements(lists)
if result:
    print("There are lists with common elements.")
else:
    print("No lists share any elements.")

There are lists with common elements.


In [13]:
def find_common_elements(lists):
    common_elements = []
    for i in range(len(lists)):
        for j in range(i + 1, len(lists)):
            common = [elem for elem in lists[i] if elem in lists[j]]
            if common:
                common_elements.append((lists[i], lists[j], i, j, common))
    return common_elements

# Test the function
lists = [[1, 2, 3], [4, 5, 6], [7, 8, 9], [3, 10], [1, 7, 8, 3]]
common_lists = find_common_elements(lists)

if common_lists:
    print("The following lists share elements:")
    for common_list in common_lists:
        list1, list2, i, j, common_elements = common_list
        print(f"{list1} and {list2}: {common_elements}, index: {i} and {j}")
else:
    print("No lists share any elements.")

The following lists share elements:
[1, 2, 3] and [3, 10]: [3], index: 0 and 3
[1, 2, 3] and [1, 7, 8, 3]: [1, 3], index: 0 and 4
[7, 8, 9] and [1, 7, 8, 3]: [7, 8], index: 2 and 4
[3, 10] and [1, 7, 8, 3]: [3], index: 3 and 4


In [27]:
common_lists = find_common_elements(list(df_de_unique["text"]))
common_lists = sorted(common_lists, key=lambda k: len(k[4]), reverse=True)
if common_lists:
    print("The following lists share elements:")
    for common_list in common_lists:
        list1, list2, i, j, common_elements = common_list
        if not i in to_delete or j in to_delete:
            to_delete.append(i)
    else:
        print("No lists share any elements.")

print(to_delete)

KeyboardInterrupt: 

In [15]:
vocab, inv = np.unique(np.concatenate(list(df_de_unique["text"])), return_inverse=True)

In [16]:
print(inv)

[ 65332  61419  87689 ...  36521 444815 400026]


In [17]:
inv.size

1454792

In [18]:
vocab

array(['                                 Das Fliegen gab mir neue Kräfte',
       '                                 Ein Vogel lud mich her zu Gaste —',
       '                                 Es schläft mir jedes Weh und Ach.',
       ..., '›zu spät zu Kampf und Beten,',
       '›zwar beendigt ist der Krieg,',
       '›über Toren, wo die Wächter liegen quer,'], dtype='<U104')

In [19]:
len(vocab)

789369

In [20]:
quads = np.resize(inv, (int(inv.size/4), 4))

In [21]:
print(quads)

[[ 65332  61419  87689 696034]
 [752366 143559 583505 275851]
 [557935 672033 411953  37402]
 ...
 [496711 254275 593174 531678]
 [708638 519395 620721 708518]
 [303874  36521 444815 400026]]


In [72]:
len(quads)

363698

43200

In [22]:
def find_overlaps(arr):
    overlaps = []
    for i in range(len(arr)):
        for j in range(i + 1, len(arr)):
            overlap_indices = np.where(np.in1d(arr[i], arr[j]))[0]
            if len(overlap_indices) > 0:
                overlaps.append((i, j, overlap_indices))
    return overlaps

# Test the function
if __name__ == "__main__":
    arr = np.array([
        [1, 2, 3, 4],
        [3, 4, 5, 6],
        [7, 8, 9, 10],
        [1, 4, 7, 10]
    ])

    overlaps = find_overlaps(arr)
    for overlap in overlaps:
        print("Overlap between arrays {} and {}: indices {}".format(overlap[0], overlap[1], overlap[2]))

Overlap between arrays 0 and 1: indices [2 3]
Overlap between arrays 0 and 3: indices [0 3]
Overlap between arrays 1 and 3: indices [1]
Overlap between arrays 2 and 3: indices [0 3]


In [61]:
print(overlaps)

[(0, 1, array([2, 3], dtype=int64)), (0, 3, array([0, 3], dtype=int64)), (1, 3, array([1], dtype=int64)), (2, 3, array([0, 3], dtype=int64))]


In [62]:
overlaps_ordered = sorted(overlaps, key=lambda k: len(k[2]), reverse=True)

print(overlaps_ordered)

[(0, 1, array([2, 3], dtype=int64)), (0, 3, array([0, 3], dtype=int64)), (2, 3, array([0, 3], dtype=int64)), (1, 3, array([1], dtype=int64))]


In [64]:
to_delete

[0, 2, 1]

In [66]:
overlaps_de = find_overlaps(quads[:1000])

In [67]:
overlaps_ordered = sorted(overlaps_de, key=lambda k: len(k[2]), reverse=True)
to_delete = []
for i, j, o in overlaps_ordered:
    if not i in to_delete or j in to_delete:
        to_delete.append(i)

In [68]:
print(to_delete)

[167, 713, 774, 922, 186, 187, 188, 189, 190, 191, 192, 316, 412]


In [71]:
def find_overlaps(arr):
    # Flatten the input array to work with 1D arrays
    flat_arr = np.vstack(arr)
    
    # Create an array of unique elements and their counts
    unique_elements, counts = np.unique(flat_arr, return_counts=True)
    
    # Find elements that appear in more than one array
    overlapping_elements = unique_elements[counts > 1]
    
    # Initialize a list to store the overlaps
    overlaps = []
    
    # Iterate through each overlapping element
    for element in overlapping_elements:
        # Find the indices of arrays that contain the element
        array_indices = [i for i, a in enumerate(arr) if np.isin(element, a)]
        print("array indices: ", array_indices)
        # Append the indices to the overlaps list
        overlaps.extend((i, j, element) for i in array_indices for j in array_indices if i < j)
    
    return overlaps

# Test the function
if __name__ == "__main__":
    arr = np.array([
        [1, 2, 3, 4],
        [3, 4, 5, 6],
        [7, 8, 9, 10],
        [1, 4, 7, 10]
    ])

    overlaps = find_overlaps(arr)
    for overlap in overlaps:
        print("Overlap between arrays {} and {}: indices {}".format(overlap[0], overlap[1], overlap[2]))

array indices:  [0, 3]
array indices:  [0, 1]
array indices:  [0, 1, 3]
array indices:  [2, 3]
array indices:  [2, 3]
Overlap between arrays 0 and 3: indices 1
Overlap between arrays 0 and 1: indices 3
Overlap between arrays 0 and 1: indices 4
Overlap between arrays 0 and 3: indices 4
Overlap between arrays 1 and 3: indices 4
Overlap between arrays 2 and 3: indices 7
Overlap between arrays 2 and 3: indices 10


In [78]:
def get_array_indices(arr):
    # Flatten the input array to work with 1D arrays
    flat_arr = np.vstack(arr)
    
    # Create an array of unique elements and their counts
    unique_elements, counts = np.unique(flat_arr, return_counts=True)
    
    # Find elements that appear in more than one array
    overlapping_elements = unique_elements[counts > 1]
    
    # Initialize a list to store the overlaps
    indices = []
    
    # Iterate through each overlapping element
    for element in overlapping_elements:
        # Find the indices of arrays that contain the element
        array_indices = [i for i, a in enumerate(arr) if np.isin(element, a)]
        indices.append(array_indices)
    
    return indices

In [80]:
get_array_indices(quads[:5000])

[[3552, 3554],
 [3365, 3366],
 [4433, 4434],
 [3880, 3881],
 [167, 168],
 [588, 1719],
 [922, 923],
 [1321],
 [713, 714],
 [4421],
 [2878, 2889],
 [2844, 2846, 2848],
 [3541, 3549],
 [4221, 4222],
 [1154, 1155, 1156, 1157],
 [186, 187, 188, 189, 190, 191, 192, 193],
 [793, 1334, 1524, 1645],
 [2306, 2307],
 [4502, 4503],
 [3541, 3549],
 [729, 1668],
 [821, 4655],
 [4969, 4970],
 [4969, 4970],
 [774, 775],
 [1116, 1122],
 [1011, 1012],
 [4382, 4383],
 [4502, 4503],
 [3788, 3789],
 [2307, 2308],
 [3537, 3538, 3539],
 [1490, 1684],
 [1125, 1291],
 [1244, 1245, 1246, 1247, 1248],
 [1242, 1243],
 [2835, 2836],
 [3044, 3963],
 [4434, 4435],
 [4451],
 [4434, 4435],
 [4426, 4427],
 [3607, 3608],
 [3605, 3606, 3607, 3608],
 [1127, 1296],
 [1423, 1424],
 [1125, 1291],
 [922, 923],
 [1125, 1291],
 [1423, 1424],
 [713, 714],
 [4011, 4063],
 [774, 775],
 [2878, 2889],
 [2878, 2889],
 [3660],
 [3500, 3502, 3503],
 [3006, 3013],
 [4382, 4383],
 [990, 1219],
 [1011, 1012],
 [412, 413],
 [1524, 1525],


In [75]:
quads

array([[ 65332,  61419,  87689, 696034],
       [752366, 143559, 583505, 275851],
       [557935, 672033, 411953,  37402],
       ...,
       [496711, 254275, 593174, 531678],
       [708638, 519395, 620721, 708518],
       [303874,  36521, 444815, 400026]], dtype=int64)

In [77]:
len(quads)

363698

In [81]:
def get_unique_elements(arr):
    # Flatten the input array to work with 1D arrays
    flat_arr = np.vstack(arr)
    
    # Create an array of unique elements and their counts
    unique_elements, counts = np.unique(flat_arr, return_counts=True)
    return(unique_elements, counts)

In [87]:
unique_elements, counts = get_unique_elements(quads)
overlapping_elements = unique_elements[counts > 1]


NameError: name 'get_unique_elements' is not defined

In [86]:
len(unique_elements)

NameError: name 'unique_elements' is not defined

In [87]:
len(overlapping_elements)

625209

In [90]:
overlapping_elements

array([    13,     27,     37, ..., 789366, 789367, 789368], dtype=int64)

In [46]:
np.isin(np.array([13, 24, 37, 39]), overlapping_elements)

NameError: name 'overlapping_elements' is not defined

In [91]:
# Iterate through each overlapping element
indices = []
for element in overlapping_elements:
    # Find the indices of arrays that contain the element
    array_indices = [i for i, a in enumerate(quads) if np.isin(element, a)]
    indices.append(array_indices)
    

KeyboardInterrupt: 

In [83]:
unique_elements

array([     0,      1,      2, ..., 789366, 789367, 789368], dtype=int64)

In [85]:
len(unique_elements)

789369

In [23]:
import numpy as np

def find_overlaps(arr):
    # Flatten the input array to work with 1D arrays
    flat_arr = np.vstack(arr)

    # Create a boolean mask of elements that appear more than once
    mask = np.isin(flat_arr, np.unique(flat_arr)[np.unique(flat_arr, return_counts=True)[1] > 1])
    #print("mask:", mask)
    print("mask size:", mask.size)

    # Reshape the mask back to the original shape
    mask_reshaped = mask.reshape(arr.shape)

    # Find the indices where the mask is True along the last axis
    overlaps_indices = np.argwhere(mask_reshaped)
    #print("overlaps_indices:", overlaps_indices)
    print("size: ", overlaps_indices.size)

    # Group the indices by the first index, indicating which array they belong to
    overlaps_groups = np.split(overlaps_indices[:, 1:], np.cumsum(np.unique(overlaps_indices[:, 0], return_counts=True)[1])[:-1])

    # Create a list of overlaps tuples
    overlaps = [(i, j, indices.flatten()) for i, indices_group in enumerate(overlaps_groups) for j, indices in enumerate(overlaps_groups) if i < j]

    return overlaps

# Test the function
if __name__ == "__main__":

    overlaps = find_overlaps(quads)
    print(len(overlaps))
    #print("Overlap between arrays {} and {}: indices {}".format(overlap[0], overlap[1], overlap[2]))
        

mask: [[False False False False]
 [False False False False]
 [False False False False]
 ...
 [False False False False]
 [False False False False]
 [False False False False]]
mask size: 4000
overlaps_indices: [[167   2]
 [167   3]
 [168   0]
 [168   1]
 [186   0]
 [187   0]
 [188   0]
 [189   0]
 [190   0]
 [191   0]
 [192   0]
 [193   0]
 [316   3]
 [317   0]
 [412   3]
 [413   0]
 [713   2]
 [713   3]
 [714   0]
 [714   1]
 [774   2]
 [774   3]
 [775   1]
 [775   2]
 [922   2]
 [922   3]
 [923   0]
 [923   1]]
size:  56
Overlap between arrays 0 and 1: indices [0 1]
Overlap between arrays 0 and 2: indices [0]
Overlap between arrays 0 and 3: indices [0]
Overlap between arrays 0 and 4: indices [0]
Overlap between arrays 0 and 5: indices [0]
Overlap between arrays 0 and 6: indices [0]
Overlap between arrays 0 and 7: indices [0]
Overlap between arrays 0 and 8: indices [0]
Overlap between arrays 0 and 9: indices [0]
Overlap between arrays 0 and 10: indices [3]
Overlap between arrays 0 and 1

In [26]:
flat_arr = np.vstack(quads)

# Create a boolean mask of elements that appear more than once
mask = np.isin(flat_arr, np.unique(flat_arr)[np.unique(flat_arr, return_counts=True)[1] > 1])
#print("mask:", mask)
print("mask size:", mask.size)

# Reshape the mask back to the original shape
mask_reshaped = mask.reshape(quads.shape)

# Find the indices where the mask is True along the last axis
overlaps_indices = np.argwhere(mask_reshaped)
#print("overlaps_indices:", overlaps_indices)
print("size: ", overlaps_indices.size)

# Group the indices by the first index, indicating which array they belong to
overlaps_groups = np.split(overlaps_indices[:, 1:], np.cumsum(np.unique(overlaps_indices[:, 0], return_counts=True)[1])[:-1])


mask size: 1454792
size:  2581264


In [44]:
tokens, counts = np.unique(flat_arr, return_counts=True)
multiples = counts > 1 
print(multiples)
multiple_tokens = tokens[multiples]
print(multiple_tokens)

[False False False ...  True  True  True]
[    13     27     37 ... 789366 789367 789368]


In [88]:
len(tokens)

789369

In [45]:
len(multiple_tokens)

625209

In [53]:
mask = np.isin(flat_arr, multiple_tokens)

In [56]:
flat_arr = np.vstack(quads)
flat_arr

array([[ 65332,  61419,  87689, 696034],
       [752366, 143559, 583505, 275851],
       [557935, 672033, 411953,  37402],
       ...,
       [496711, 254275, 593174, 531678],
       [708638, 519395, 620721, 708518],
       [303874,  36521, 444815, 400026]], dtype=int64)

In [54]:
mask

array([[False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       ...,
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [65]:
array_sums = np.array([ar.sum() for ar in mask])


In [126]:
smaller_four = array_sums < 4
np.argwhere(smaller_four).flatten()

array([     0,      1,      2, ..., 169319, 169320, 169321], dtype=int64)

In [95]:
equals_four = array_sums==4
equals_four_indices = np.argwhere(equals_four).flatten()

In [127]:
import random 
equals_four_indices_sample = random.choices(equals_four_indices, k = int(equals_four_indices.size/2))

In [128]:
len(equals_four_indices)

320498

In [129]:
len(equals_four_indices_sample)

160249

In [130]:
to_keep = smaller_four.copy()
to_keep[equals_four_indices_sample] = True

IndexError: index 230994 is out of bounds for axis 0 with size 169322

In [103]:
smaller_four.sum()

43200

In [102]:
to_keep.sum()

169322

In [104]:
quads_to_keep = quads[to_keep]

In [107]:
new_elements, new_counts = np.unique(quads_to_keep, return_counts=True)
new_elements.size

565890

In [118]:
x = new_counts > 1 

In [122]:
x

array([False, False, False, ..., False, False, False])

In [119]:
x.sum()

106195

In [123]:
mask = np.isin(quads_to_keep, new_elements[new_counts > 1])

In [124]:
mask

array([[False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       ...,
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False]])

In [125]:
array_sums = np.array([ar.sum() for ar in mask])
still_equals_four = array_sums==4
still_equals_four.sum()

50632

In [113]:
equals_one = array_sums==1
equals_one.sum()

2

In [114]:
(np.unique(quads_to_keep, return_counts=True)[1] > 1).sum()

106195

In [75]:
smaller_four.sum()

43200

In [71]:
len(smaller_four)

363698

In [77]:
quads_smaller_four = quads[smaller_four]


In [83]:
(np.unique(quads_smaller_four, return_counts=True)[1] > 1).sum()

1111

In [85]:
new_elements, new_counts = np.unique(quads_smaller_four, return_counts=True)
new_elements.size

171519

In [35]:
quads

array([[ 65332,  61419,  87689, 696034],
       [752366, 143559, 583505, 275851],
       [557935, 672033, 411953,  37402],
       ...,
       [496711, 254275, 593174, 531678],
       [708638, 519395, 620721, 708518],
       [303874,  36521, 444815, 400026]], dtype=int64)

In [36]:
print(quads * mask_reshaped)

[[     0      0      0      0]
 [     0      0      0      0]
 [     0      0      0      0]
 ...
 [496711 254275 593174 531678]
 [708638 519395 620721 708518]
 [303874  36521 444815 400026]]


In [28]:
len(overlaps_groups)

324748

In [30]:
print(overlaps_groups[:10])

[array([[2],
       [3]], dtype=int64), array([[0],
       [1]], dtype=int64), array([[0]], dtype=int64), array([[0]], dtype=int64), array([[0]], dtype=int64), array([[0]], dtype=int64), array([[0]], dtype=int64), array([[0]], dtype=int64), array([[0]], dtype=int64), array([[0]], dtype=int64)]
